In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import sqlite3

# 1. Làm quen SQL

In [ ]:
# Kết nối đế CSDL
conn = sqlite3.connect('database/Chinook_Sqlite.sqlite')

In [ ]:
# Tạo cursor để thực thi lệnh truy vấn
cursor = conn.cursor()

In [ ]:
# Thực thi lệnh và gọi hàm fetchall để trả về toàn bộ kết quả
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
res = cursor.fetchall()

In [ ]:
print(res)

In [ ]:
# Truy vấn thông tin của một bảng nào đó
cursor.execute("PRAGMA table_info(Employee)")
info = cursor.fetchall()

In [ ]:
print(*info, sep='\n')

In [ ]:
# Lấy ra thông tin mô tả chung của truy vấn hiện tại
cursor.description

In [ ]:
# Biến thông tin đã truy vấn thành dataframe của pandas
pd.DataFrame(info, columns= [item[0] for item in cursor.description])

In [ ]:
# Truy vấn thử thông tin từ 1 bảng nào đó
cursor.execute("SELECT EmployeeId, Firstname, Lastname FROM Employee LIMIT(10)")
cursor.fetchall()

In [ ]:
# Tạo câu truy vấn (dễ kiểm tra hơn)
query = """
        SELECT EmployeeId, Firstname, Lastname, HireDate, Country 
        FROM Employee 
        WHERE Country != 'USA'
        LIMIT(20)
"""
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[item[0] for item in cursor.description])

In [ ]:
df.sample(5)

In [ ]:
# Truy vấn với sự sắp xếp theo 1 cột nào đó
query = """
        SELECT EmployeeId, Firstname, Lastname, HireDate, Country 
        FROM Employee 
        WHERE Country != 'USA'
        ORDER BY HireDate Desc
        LIMIT(20)
"""
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[item[0] for item in cursor.description])
df.head(5)

In [ ]:
# Truy vấn với việc chuyển đổi text sang Date
query = """
        SELECT EmployeeId, Firstname, Lastname, DATE(HireDate) as HireDate, Country 
        FROM Employee 
        WHERE Country != 'USA'
        ORDER BY HireDate Desc
        LIMIT(20)
"""
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[item[0] for item in cursor.description])
df.head(5)

In [ ]:
# Truy vấn với ràng buộc về ngày
query = """
        SELECT EmployeeId, Firstname, Lastname, DATE(HireDate) as HireDate, Country 
        FROM Employee 
        WHERE Country != 'USA' AND HireDate BETWEEN '2003-05-01' AND '2004-01-31'
        ORDER BY HireDate Desc
        LIMIT(20)
"""
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[item[0] for item in cursor.description])
df.head(5)

# Demo với SQLite

## (1) Kết nối database

In [84]:
# db = sqlite3.connect(':memory:')
con = sqlite3.connect('database/example.db')

In [85]:
cur = con.cursor()

## (2) Một số lệnh đơn giản

**Tạo bảng PhongBan**

In [98]:
query = """
        CREATE TABLE phongban (id INTEGER,
                               ten VARCHAR(40) NOT NULL, id_truongphong INTEGER,
                               PRIMARY KEY (id)
                               )
"""
cur.execute(query)
con.commit()

In [97]:
# Thử xóa phòng ban
cur.execute("DROP TABLE IF EXISTS phongban")
con.commit()

In [99]:
# Thêm data vào bảng phongban
query = """
        INSERT INTO phongban (ten)
        VALUES('Phong Nhan Su')
"""
cur.execute(query)
con.commit()

In [82]:
#con.close()

In [100]:
query = """
        INSERT INTO phongban 
        VALUES(2, 'Phong Ke Toan', NULL)
"""
cur.execute(query)
con.commit()

In [101]:
query = """
        INSERT INTO phongban (ten)
        VALUES('Phong Phong San Xuat')
"""
cur.execute(query)
con.commit()

In [103]:
# Thêm một list các phòng ban bằng lệnh executemany 
phongban_list = [
    ('Phong Kinh Doanh - Marketing', None),
    ('Phong Ky Thuat', None),
    ('Phong R&D', None)
]
cur.executemany("insert into phongban (ten, id_truongphong) values(?,?)", phongban_list)
con.commit()

**Tạo bảng NhanVien**

In [91]:
query = """
        CREATE TABLE nhanvien (id INTEGER PRIMARY KEY,
                               ho TEXT NOT NULL, ten TEXT NOT NULL,
                               email TEXT NOT NULL UNIQUE,
                               id_phongban INTEGER NOT NULL, 
                               id_quanly INTEGER UNIQUE,
                               ngay_batdau DATETIME,
                               luong REAL NOT NULL)
"""
cur.execute(query)
con.commit()

In [105]:
query = """
        INSERT INTO nhanvien (ho, ten, email, id_phongban, luong)
        VALUES('Tran', 'Van A', 'atran@gmail.com', 3, 15000)
"""
cur.execute(query)
con.commit()

In [108]:
query = """
        INSERT INTO nhanvien (ho, ten, email, id_phongban, luong)
        VALUES('Nguyen', 'Van B', 'bnguyen@gmail.com', 3, 11000)
"""
cur.execute(query)
con.commit()

**Update thông tin cho các bảng sau khi thêm được thông tin nhân viên**

In [109]:
cur.execute("SELECT * FROM nhanvien WHERE id_phongban=3")
cur.fetchall()

[(1, 'Tran', 'Van A', 'atran@gmail.com', 3, None, None, 15000.0),
 (2, 'Nguyen', 'Van B', 'bnguyen@gmail.com', 3, None, None, 11000.0)]

In [110]:
# Update trưởng phòng cho phòng id=3
query = """
        UPDATE phongban
        SET id_truongphong=2
        WHERE id=3
"""
cur.execute(query)
con.commit()

In [111]:
# Update người quản lý cho nhân viên có id=1
query = """
        UPDATE nhanvien
        SET id_quanly=2
        WHERE id=1
"""
cur.execute(query)
con.commit()